4. Build a spam classifier (a more challenging exercise):

    • Download examples of spam and ham from Apache SpamAssassin’s public
    datasets.

    • Unzip the datasets and familiarize yourself with the data format.

    • Split the datasets into a training set and a test set.
    
    • Write a data preparation pipeline to convert each email into a feature vector.
    Your preparation pipeline should transform an email into a (sparse) vector that
    indicates the presence or absence of each possible word. For example, if all
    emails only ever contain four words, “Hello,” “how,” “are,” “you,” then the email
    “Hello you Hello Hello you” would be converted into a vector [1, 0, 0, 1]
    (meaning [“Hello” is present, “how” is absent, “are” is absent, “you” is
    present]), or [3, 0, 0, 2] if you prefer to count the number of occurrences of
    each word.
    
    You may want to add hyperparameters to your preparation pipeline to control
    whether or not to strip off email headers, convert each email to lowercase,
    remove punctuation, replace all URLs with “URL,” replace all numbers with
    “NUMBER,” or even perform stemming (i.e., trim off word endings; there are
    Python libraries available to do this).
    Finally, try out several classifiers and see if you can build a great spam classi
    fier, with both high recall and high precision.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import os
from pathlib import Path

ham_path = os.path.join(os.getcwd(), "easy_ham")
spam_path = os.path.join(os.getcwd(), "spam")

In [3]:
def is_email_file(path):
    # Check filename starts with digit and content has 'Subject:' header
    if path.is_file() and path.name[0].isdigit():
        text = path.read_text(encoding="latin1")
        if "Subject:" in text or "From:" in text:
            return True
    return False

def load_emails(folder):
    email_dir = Path(folder)
    emails = []

    # iterdir() returns a list of Path objects in the email_dir
    for path in email_dir.iterdir():
        if is_email_file(path):
            try:
                text = path.read_text(encoding="latin1")
                emails.append(text)
            except:
                print(f"Skipping file {path.name}")

        else:
            print(f"{path.name} is not an email file")

    return emails

In [4]:
spam_emails = load_emails(spam_path)
ham_emails = load_emails(ham_path)

0000.7b1b73cf36cf9dbc3d64e3f2ee2b91f1 is not an email file


In [5]:
print(len(spam_emails))
print(len(ham_emails))

500
2551


In [6]:
data = [(email_str, 1) for email_str in spam_emails]
data += [(email_str, 0) for email_str in ham_emails]

len(data)

3051

# BETTER URL HANDLING NEEDED

In [7]:
from email.parser import Parser
from email import policy 
from bs4 import BeautifulSoup
import regex as re

# Preprocesser
class PreProcessing(BaseEstimator, TransformerMixin): 
    
    def __init__(self, strip_headers=True, lowercase=True, remove_punc=True,
                 replace_url=True, remove_numbers=True):

        self.strip_headers = strip_headers
        self.lowercase = lowercase
        self.remove_punc = remove_punc
        self.replace_url = replace_url
        self.remove_numbers = remove_numbers
    

    def __html_to_text(self, html):
        soup = BeautifulSoup(html, "html.parser")
        return soup.get_text()

    # Merges header with the email's text/plain content
    def __merge_header_content(self, text_plain, email_msg_obj):
        email_with_header = ""

        for item in email_msg_obj.items():
            email_with_header += f"{item[0]}: {item[1]}\n"

        email_with_header = email_with_header + text_plain

        return email_with_header  
    
    # Returns string, EmailMessage
    def __extract_email(self, email_str):
        # Instantiates an EmailMessage object from string value
        email_message = Parser(policy=policy.default).parsestr(email_str)       

        # if msg's mail is singlepart returns content as string
        if email_message.is_multipart() is False:
            content_type = email_message.get_content_type()

            charset = email_message.get_content_charset()
            if not charset:
                charset = "latin1"

            # Each part is decoded to byte strings
            payload = email_message.get_payload(decode=True)
            payload = payload if payload is not None else b""

            if isinstance(payload, bytes):
                try:
                    # According to part's charset, byte string is decoded to Unicode Python string
                    text = payload.decode(charset, errors="replace")
                except LookupError:
                    charset = "latin1"
                    text = payload.decode(charset, errors="replace")
            else:
                text = payload

            text = text or ""

            # if content type is html, it is converted to text 
            text = self.__html_to_text(text) if content_type == "text/html" else text

            return text, email_message
                
        # else returns only text/plain part's content as string
        else:
            text_html = ""

            for part in email_message.iter_parts():
                content_type = part.get_content_type()

                charset = part.get_content_charset()
                if not charset:
                    charset = "latin1"

                # Each part is decoded to byte strings
                payload = part.get_payload(decode=True)
                payload = payload if payload is not None else b""

                if isinstance(payload, bytes):
                    try:
                        # According to part's charset, byte string is decoded to Unicode Python string
                        text = payload.decode(charset, errors="replace")
                    except LookupError:
                        charset = "latin1"
                        text = payload.decode(charset, errors="replace")
                else:
                    text = payload

                text = text or ""

                if content_type == "text/plain":
                    return text, email_message
                elif content_type == "text/html":
                    text_html = self.__html_to_text(text)

            # if no text/plain it either falls back to html, if both are empty returns empty
            return text_html, email_message


    def __remove_punc(self, email_str):
        # \p{P} matches any kind of punctuation character from any language
        return re.sub(r'\p{P}+', '', email_str)

    def __remove_numbers(self, email_str):
        return re.sub(r'\b\d+(?:\.\d+)?\b', 'NUMBER', email_str)

    # TODO: BETTER URL HANDLING NEEDED
    def __remove_url(self, email_str):
        return re.sub(r'https?://\S+|www\.\S+', 'URL', email_str)
       
    

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        processed_X = []

        # data: [(email_str, label), (email_str, label)...]
        for email_str in X:

            text_plain, email_message = self.__extract_email(email_str)

            if self.strip_headers:
                email_str = text_plain
            else:
                email_str = self.__merge_header_content(text_plain, email_message)

            if self.lowercase:
                email_str = email_str.lower()
            if self.replace_url:
                email_str = self.__remove_url(email_str)
            
            if self.remove_punc:
                email_str = self.__remove_punc(email_str)        

            if self.remove_numbers:
                email_str = self.__remove_numbers(email_str)            


            processed_X.append(email_str)
        
        return processed_X
    

preprocesser = PreProcessing()

In [8]:
import joblib

In [9]:
import nltk
from nltk.corpus import stopwords

In [10]:
from nltk.stem import PorterStemmer

In [11]:
from scipy.sparse import csr_matrix

In [ ]:
stop_words =set(stopwords.words("english"))

class WordCountVectorizer(BaseEstimator, TransformerMixin):

    def __init__(self, stemming=True, vocab_size=5000, sparse=True):
        self.stop_words = stop_words # Here made a change for n_jobs=-1
        self.stemming = stemming
        self.vocab_size = vocab_size
        self.sparse = sparse
    
    def __create_vocab(self, X):
        token_list = []
        X_tokens = []

        stop_words = self.stop_words
        # PorterStemmer behaves determinestically
        stemmer = PorterStemmer()
        if self.stemming:
            stop_words = [stemmer.stem(word) for word in stop_words]

        for email_str in X:
            email_str = email_str.replace("\n", " ")
            email_str = email_str.strip()
            email_str = self.__subs_datetime(email_str)

            row_tokens = [token for token in re.findall(r"\w+|[^\w\s]", email_str)]
            if self.stemming:
                row_tokens = [stemmer.stem(token, to_lowercase=False) for token in row_tokens]

            row_tokens = [self.__validate_and_tokenize(token, stop_words) for token in row_tokens]
            row_tokens = [token for token in row_tokens if token is not None]

            X_tokens.append(row_tokens)
            token_list.extend(row_tokens)

        # Our vocab_size will determine feauture counts, if remaining tokens are not in this list
        # they will be counted as OTHER_ELEMENT
        features = pd.Series(token_list).value_counts(ascending=False).index.to_list()
        if self.vocab_size > len(features) or self.vocab_size=="max":
            pass
        else:
            features = features[:self.vocab_size]

        # X_tokens is X, contains list of tokens of each row
        return features, X_tokens


    
    def __count_matrixation(self, X, features):
        # X is X_tokens, 26000 rows
        email_dict = {}
        count_dict_base = {feature : 0 for feature in features}   # each key of email_dict will contain a dictionary for count dict for every feature
        count_dict_base["OTHER_ELEMENT"] = 0

        for idx, email_tokens in enumerate(X):
            tokens_value_counts = pd.Series(email_tokens).value_counts().to_dict()
            count_dict = count_dict_base.copy()

            for key, value in tokens_value_counts.items():
                if key in features:
                    count_dict[key] = value
                else:
                    count_dict["OTHER_ELEMENT"] += value

            email_dict[f"email{idx}"] = count_dict       

        email_dict_df = pd.DataFrame(email_dict).T
        X_sparse_matrix = csr_matrix(email_dict_df.values)
        
        if self.sparse:
            return X_sparse_matrix
        else:
            return email_dict_df

    
    def __validate_and_tokenize(self, token, stop_words):
        token = token.strip()
        if len(token) < 20 and len(token) > 0 and token.lower() not in stop_words:
            return token
        


    def __subs_datetime(self, email_str):
        datetime_regex = r'''
            \b
            (                                # group entire match
                \d{8}[Tt]?\d{6}              # 20021009T225035 or 20021009t205951
                (?:Z|[+-]?\d{4})?            # optional timezone like Z or +0800
                |
                \d{4}[-./]\d{2}[-./]\d{2}    # 2002-12-01, 2002/12/01, 2002.12.01
                (?:[Tt]\d{2}[:.]?\d{2}[:.]?\d{2})?  # optional time like T23:45:12
            )
            \b
        '''
        return re.sub(datetime_regex, 'DATETIME', email_str, flags=re.IGNORECASE | re.VERBOSE)



    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        columns, X_tokens = self.__create_vocab(X)
        X_matrix = self.__count_matrixation(X_tokens, columns)
        return X_matrix

vector_transformer = WordCountVectorizer()

## ML Implementation

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV

pipeline = Pipeline([
    ("preprocess", PreProcessing()),
    ("vectorizer", WordCountVectorizer()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

In [69]:
X_raw, y = zip(*data)

In [70]:
param_dist = {
    "preprocess__strip_headers": [True, False],
    "preprocess__lowercase": [True, False],
    "preprocess__remove_punc": [True],
    "preprocess__replace_url": [True],
    "preprocess__remove_numbers": [True],

    "vectorizer__stemming": [True],
    "vectorizer__vocab_size": [25000],
    "vectorizer__sparse": [True],

    "clf__C": [0.1, 1.0, 10.0]
}

X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42)

In [71]:
from sklearn.model_selection import StratifiedKFold

stratifed_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [72]:
randomized_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,  # rename for clarity
    n_iter=1,  # number of hyperparameter combinations
    cv=stratifed_cv,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring="recall"
)
randomized_search.fit(X_train, y_train)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\talha\miniconda3\envs\hands_on_ml\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('preprocess', PreProcessing()),
                                             ('vectorizer',
                                              WordCountVectorizer()),
                                             ('clf',
                                              LogisticRegression(class_weight='balanced',
                                                                 max_iter=1000))]),
                   n_iter=1, n_jobs=-1,
                   param_distributions={'clf__C': [0.1, 1.0, 10.0],
                                        'preprocess__lowercase': [True, False],
                                        'preprocess__remove_numbers': [True],
                                        'preprocess__remove_punc': [True],
                                        'preprocess__replace_url': [True],
                                        'preprocess__strip_headers': [True,
                                                                      False],
                                        'vectorizer__sparse': [True],
                                        'vectorizer__stemming': [True],
                                        'vectorizer__vocab_size': [25000]},
                   random_state=42, scoring='recall', verbose=2)

In [73]:

print("Best parameters:", randomized_search.best_params_)
print("Best cross-validation score:", randomized_search.best_score_)
print("Test set accuracy:", randomized_search.score(X_test, y_test))


Best parameters: {'vectorizer__vocab_size': 25000, 'vectorizer__stemming': True, 'vectorizer__sparse': True, 'preprocess__strip_headers': True, 'preprocess__replace_url': True, 'preprocess__remove_punc': True, 'preprocess__remove_numbers': True, 'preprocess__lowercase': False, 'clf__C': 10.0}
Best cross-validation score: nan


ValueError: X has 14351 features, but LogisticRegression is expecting 25001 features as input.

In [40]:
best_model = randomized_search.best_estimator_

best_model

Pipeline(steps=[('preprocess', PreProcessing(lowercase=False)),
                ('vectorizer', WordCountVectorizer()),
                ('clf',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    max_iter=1000))])

In [41]:
best_pred = best_model.predict(X_test)

In [42]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(f1_score(y_test, best_pred))
print(precision_score(y_test, best_pred))
print(recall_score(y_test, best_pred))

0.10582010582010581
0.13333333333333333
0.08771929824561403


## Bug Fix

In [46]:
data_test = [(email_str, 1) for email_str in spam_emails]
data_test += [(email_str, 0) for email_str in ham_emails]

len(data_test)

3051

In [47]:
preprocesser_test = PreProcessing()

In [55]:
processed = preprocesser_test.transform(spam_emails)
processed

['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsave up to NUMBER on life insurance\nwhy spend more than you have to\n\nlife quote savings\n\n\n\n\n\n\n\n\n\n\nensuring your \n      familys financial security is very important life quote savings makes \n      buying life insurance simple and affordable we provide free access to the \n      very best companies and the lowest rates\n\n\n\n\n\nlife quote savings is fast easy and \n            saves you money let us help you get started with the best values in \n            the country on new coverage you can save hundreds or even thousands \n            of dollars by requesting a free quote from lifequote savings our \n            service will take you less than NUMBER minutes to complete shop and \n            compare save up to NUMBER on all types of life insurance \n\n\n\nclick here for your \n            free quote\n\nprotecting your family is the best investment youll ever \n          make\n\n\n\n\n\n\n\n\nif you are in receipt of this email \n     

In [60]:
vectorizer_test = WordCountVectorizer(vocab_size=20000, sparse=False)

In [61]:
vectorizer_test.transform(processed)

,NUMBER,=,+,$,email,�,URL,free,receiv,money,...,botanicalherb,precursorship,taoist,virpot,breakthrough,xbr,brewtea,bowl,partak,OTHER_ELEMENT
email0,3,0,0,0,3,0,0,3,0,1,...,0,0,0,0,0,0,0,0,0,0
email1,7,0,0,0,0,0,7,1,0,0,...,0,0,0,0,0,0,0,0,0,0
email2,7,0,0,0,0,0,6,1,0,0,...,0,0,0,0,0,0,0,0,0,0
email3,15,0,0,0,2,0,6,11,4,0,...,0,0,0,0,0,0,0,0,0,0
email4,5,0,0,0,0,0,5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
email495,18,0,0,6,15,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
email496,47,0,0,0,1,0,4,0,2,0,...,0,0,0,0,0,0,0,0,0,0
email497,2,0,0,0,6,0,2,0,3,0,...,0,0,0,0,0,0,0,0,0,0
email498,3,0,0,1,3,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
